In [45]:
import pandas as pd
import re

Set up tweets df

In [46]:
preprocessed_tweets = "gg2013_preprocessed.json"
tweets = pd.read_json(preprocessed_tweets, orient='records', lines=True)[["text","timestamp","is_english"]]
tweets = tweets[tweets["is_english"]] #only keep english tweets
del tweets["is_english"] #get rid of this row
tweets = tweets.dropna(subset=["text"])

In [47]:
#get hosts for entire ceremony
#get award names for the ceremony

function to get hosts

In [48]:
def get_hosts(tweets):
    host_tweets = tweets[tweets["text"].str.contains("host")] #get tweets with the word host in it
    #handle single names or no spaces?
    host_patterns = ['host (?P<name>[a-z]+ [a-z]+)', 
                     '(?P<name>[a-z]+ [a-z]+) (is hosting|will host|hosts)']
    cohost_patterns = ['(co-?|)hosts (?P<name1>[a-z]+ [a-z]+) and (?P<name2>[a-z]+ [a-z]+)',
                    '(?P<name1>[a-z]+ [a-z]+) and (?P<name2>[a-z]+ [a-z]+) (are (co-?|)hosting|will (co-?|)host|(co-?|)host)']
    
    

In [49]:
host_tweets = tweets[tweets["text"].str.contains("host")] #get tweets with the word host in it
#remember all tweets are lowercase
host_patterns = ['host (?P<first_name>[a-z]+) (?P<last_name>[a-z]+)',
            '(?P<first_name_alt>[a-z]+) (?P<last_name_alt>[a-z]+) (is hosting|will host|hosts)',
            '(co|)hosts (?P<cohost1_first_name>[a-z]+) (?P<cohost1_last_name>[a-z]+) and (?P<cohost2_first_name>[a-z]+) (?P<cohost2_last_name>[a-z]+)',
            '(?P<cohost1_first_name_alt>[a-z]+) (?P<cohost1_last_name_alt>[a-z]+) and (?P<cohost2_first_name_alt>[a-z]+) (?P<cohost2_last_name_alt>[a-z]+) (are hosting|will host|host)'
            ]
host_pattern = (" | ").join(host_patterns)
host_re_func = re.compile(pattern=host_pattern)
#host_tweets["text"].map(lambda x: host_re_func.search(x))
for i,ele in host_tweets.iterrows():
    t = ele["text"]
    m = host_re_func.search(t)
    if m is not None:
        print(m)



<re.Match object; span=(27, 57), match=' tina fey and amy poehler host'>
<re.Match object; span=(15, 30), match=' its our hosts '>
<re.Match object; span=(63, 78), match='host tina feys '>
<re.Match object; span=(2, 31), match=' usweekly goldenglobes hosts '>
<re.Match object; span=(2, 19), match=' afp photo hosts '>
<re.Match object; span=(6, 24), match=' red carpet hosts '>
<re.Match object; span=(2, 19), match=' afp photo hosts '>
<re.Match object; span=(72, 109), match=' tina fey and amy poehler are hosting'>
<re.Match object; span=(78, 94), match='host love those '>
<re.Match object; span=(24, 54), match=' tina fey and amy poehler host'>
<re.Match object; span=(13, 50), match=' tina fey and amy poehler are hosting'>
<re.Match object; span=(56, 70), match='host does not '>
<re.Match object; span=(18, 49), match=' tina fey and amy poehlers host'>
<re.Match object; span=(69, 85), match='host dick under '>
<re.Match object; span=(48, 78), match=' tina fey and amy poehler host'>
<re.Ma

Function to get awards

In [70]:

def clean_and_sort_text(text):
    # function to group award names, even if they are missing a key word or a "-"
    words = re.sub(r"(motion|original|-)", "", text).split()
    # sort words and join back into a string
    return ' '.join(sorted(words))


def extract_award_names(df, text_column='text'):
    extracted_award_names = []
    # indentify which tweets might contain an award name
    keywords = ['win*', 'award', 'goes to']
    pattern = '|'.join(keywords)
    candidate_tweets = df[df[text_column].str.contains(pattern, regex=True)]

    # matching regex to all the tweets
    regex_pattern = r"\b(wins|goldenglobes|awarded|goes to) (.*?)(award)?($|[,.!?:;])"
    for tweet in candidate_tweets[text_column]:
        matches = re.findall(regex_pattern, tweet)
        for match in matches:
            award_name = match[1].strip()
            extracted_award_names.append(award_name)

    # clean up the results so we can rank them
    delimiters = r"( http*| at *| odds-on *| goldenglobes*| and *| goes *)"
    cleaned_award_names = [re.split(delimiters, name, 1)[0] for name in extracted_award_names]
    cleaned_award_names = ["-".join(name.split("-", 2)[:2]).strip() for name in cleaned_award_names if name.lower().startswith('best ') and len(name.split()) > 2]
    cleaned_award_df = pd.DataFrame(cleaned_award_names, columns=['Cleaned_Award_Name'])
    cleaned_award_counts = cleaned_award_df['Cleaned_Award_Name'].value_counts().reset_index()
    cleaned_award_counts.columns = ['Cleaned_Award_Name', 'Frequency']

    cleaned_award_counts['Sorted_Award_Name'] = cleaned_award_counts['Cleaned_Award_Name'].apply(clean_and_sort_text)
    
    # chooses the longest award name of the grouping
    def prefer_longer(series):
        return max(series, key=len)

    # group by the cleaned and sorted award names and add the frequencies together
    grouped_award_counts = cleaned_award_counts.groupby('Sorted_Award_Name').agg({
        'Cleaned_Award_Name': prefer_longer,  # keep the longest version
        'Frequency': 'sum'  # add up their frequencies
    }).reset_index(drop=True)

    grouped_award_counts = grouped_award_counts.sort_values(by='Frequency', ascending=False).reset_index(drop=True)
    return grouped_award_counts

award_names = extract_award_names(tweets)
print(award_names)


                                    Cleaned_Award_Name  Frequency
0              best motion picture - comedy or musical         38
1                          best motion picture - drama         24
2               best actor in a motion picture - drama         20
3                               best director for argo         19
4                   best song for the theme to skyfall         12
..                                                 ...        ...
537  best actor motion picture for comedymusical fo...          1
538  best actor movie comedy or musical sing-talk t...          1
539           best actor comedy or musical for les mis          1
540  best actor in comedy or musical for les misera...          1
541                           best song unsurprisingly          1

[542 rows x 2 columns]


## Part Two
- have awards as input
- get presenters for awards
- get nominees for awards
- get winners for awards

In [71]:
awards = ["best performance by an actor in a television series - comedy or musical", "best performance by an actor in a television series - drama",
          "best television series - comedy or musical", "best performance by an actor in a motion picture - drama",
          "best original song - motion picture", "best animated feature film", "best performance by an actress in a mini-series or motion picture made for television",
          "best performance by an actor in a mini-series or motion picture made for television", "best television series - drama", "best performance by an actress in a supporting role in a motion picture",
          "best performance by an actor in a supporting role in a series, mini-series or motion picture made for television", "best motion picture - drama",
          "best performance by an actor in a motion picture - comedy or musical", "cecil b. demille award", "best performance by an actress in a motion picture - drama",
          "best performance by an actress in a television series - drama", "best original score - motion picture", "best mini-series or motion picture made for television",
          "best performance by an actress in a motion picture - comedy or musical", "best motion picture - comedy or musical", "best performance by an actress in a supporting role in a series, mini-series or motion picture made for television",
          "best performance by an actor in a supporting role in a motion picture", "best foreign language film", "best performance by an actress in a television series - comedy or musical",
          "best director - motion picture", "best screenplay - motion picture"]
count = 0
for award in award_names["Cleaned_Award_Name"]:
    if award in awards:
        print(award)
        count += 1
count

best motion picture - comedy or musical
best motion picture - drama
best animated feature film
best original song - motion picture
best screenplay - motion picture
best foreign language film
best mini-series or motion picture made for television
best original score - motion picture
best television series - drama
best performance by an actor in a motion picture - drama
best performance by an actress in a television series - drama


11